In [4]:
from semantic_text_splitter import TextSplitter
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np
import PyPDF2


In [ ]:
import os
import math
import textwrap
from typing import List, Optional

# If using the modern openai client:
# pip install openai
import openai

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Set OPENAI_API_KEY environment variable")

openai.api_key = OPENAI_API_KEY


In [45]:
KEY_FILENAME = "openai_key.txt"

In [46]:
KEY_FILENAME

'openai_key.txt'

In [9]:
import os
dir = r"C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attachments"
all_entries = os.listdir(dir)

C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attachments/EPA-HQ-OAR-2025-0194-0094_attachment_1.pdf
C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attachments/EPA-HQ-OAR-2025-0194-0094_attachment_2.pdf
C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attachments/EPA-HQ-OAR-2025-0194-0094_attachment_3.pdf
C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attachments/EPA-HQ-OAR-2025-0194-0094_attachment_4.pdf
C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attachments/EPA-HQ-OAR-2025-0194-0094_attachment_5.pdf
C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attachments/EPA-HQ-OAR-2025-0194-0094_attachment_7.pdf
C:/Users/Eric.Englin/DOT OST\Volpe-Group-JPODataProgram - ROADII/Lab/Use Cases/Public Comments/attac

In [17]:
def pdf_to_text(pdf_path, output_txt_path = None):
    '''
    Inputs
        pdf_path: file path of the PDF to be read
        output_txt_path: File path of the txt output file if you would like one. If left blank, no file will be saved.

    Returns:
        A string containing the text from the PDF
    '''
    # Open the PDF file in read-binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PdfReader object instead of PdfFileReader
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Initialize an empty string to store the text
        text = ''

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Write the extracted text to a text file
    if output_txt_path != None:
        with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

    return text

In [ ]:
def standardizeWhitespace(text):
    '''
    Takes a string and ensures all whitespace around punctuation is consistent.
    Returns string with more normal whitespace
    '''
    pass




In [44]:
# -------------------------
# Local chunker (deterministic)
# -------------------------
def chunk_by_size(text: str,
                  max_chars: int = 800,
                  prefer_sentence_boundary: bool = True) -> List[str]:
    """
    Break `text` into chunks not exceeding max_chars (approx).
    If prefer_sentence_boundary is True, tries to split on sentence boundaries.
    """
    text = text.strip()
    if not text:
        return []

    if not prefer_sentence_boundary:
        # simple fixed-size chunks (preserves words)
        words = text.split()
        chunks = []
        cur = []
        cur_len = 0
        for w in words:
            if cur_len + len(w) + (1 if cur else 0) > max_chars:
                chunks.append(" ".join(cur))
                cur = [w]
                cur_len = len(w)
            else:
                cur.append(w)
                cur_len += len(w) + (1 if cur_len else 0)
        if cur:
            chunks.append(" ".join(cur))
        return chunks

    # Prefer splitting at sentence boundaries
    # A simple heuristic-based sentence split using punctuation. For complex cases, use nltk/spacy.
    import re
    sentence_end_re = re.compile(r'(?<=[.!?])\s+')
    sentences = sentence_end_re.split(text)

    chunks = []
    cur = ""
    for s in sentences:
        s = s.strip()
        if not s:
            continue
        if len(cur) + 1 + len(s) <= max_chars:
            cur = (cur + " " + s).strip() if cur else s
        else:
            if cur:
                chunks.append(cur)
            # If single sentence is too big, fall back to word-level greedy packing
            if len(s) > max_chars:
                words = s.split()
                temp = []
                cur2 = ""
                for w in words:
                    if len(cur2) + 1 + len(w) <= max_chars:
                        cur2 = (cur2 + " " + w).strip() if cur2 else w
                    else:
                        temp.append(cur2)
                        cur2 = w
                if cur2:
                    temp.append(cur2)
                chunks.extend(temp)
                cur = ""
            else:
                cur = s
    if cur:
        chunks.append(cur)
    return chunks

NameError: name 'List' is not defined

In [26]:
def splitDocument(doctring, min_characters = 100, max_characters = 250):
    '''
    Takes a document that has already been converted to string format and divides it up into chunks to be categorized
    '''
    splitter = TextSplitter((min_characters, max_characters), trim=True)
    chunks = splitter.chunks(doctring)
    return chunks


In [ ]:
def tokenizeChunks(chunks):
    '''
    convert new data to tokenized form to be fed to NN
    '''
    vectorizer = TfidfVectorizer(
        sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
    )
    X_data = vectorizer.fit_transform(chunks)
    X_data = X_data.toarray()

    # Convert testing data format to csr_matrix for scikit
    rows = []
    cols = []
    data = []
    # test_df = test_df.reset_index()
    for i, row in tqdm(enumerate(X_data)):
        data_in_row = row
        for feature_num, val in enumerate(data_in_row):
            if val != 0:
                rows.append(i)
                cols.append(feature_num)
                data.append(val)

    row = np.array(rows)
    col = np.array(cols)
    data = np.array(data)
    tokenizedChunks = csr_matrix((data, (row, col)), shape=(i+1, len(data_in_row)))

    return tokenizedChunks

In [24]:
attachment_text = []

for x in all_entries[0:3]:
    if ".pdf" in x:
        pdf_path = dir + "/" + x
        this_attachment_text = pdf_to_text(pdf_path, output_txt_path = None)
        attachment_text.append(this_attachment_text)
    else:
        attachment_text.append("not a pdf")



In [30]:
attachment_chunks = []
attachment_doc_list = []

for x in attachment_text:
    print(splitDocument(x, min_characters = 300, max_characters = 800))

['Association between Diet-Related Greenhouse Gas Emissions and Mortality among\nJapanese Adults: The Japan Collaborative Cohort Study\nDaiki Watanabe,1,2Kotatsu Maruyama,3Akiko Tamakoshi,4Isao Muraki,5\nand the JACC Study Group\n1Faculty of Sport Sciences, Waseda University, Tokorozawa, Saitama, Japan\n2National Institute of Health and Nutrition, National Institutes of Biomedical Innovation, Health and Nutrition, Settsu, Osaka, Japan', '3Laboratory of Community Health and Nutrition, Special Course of Food and Health Science, Department of Bioscience, Graduate School of Agriculture,\nEhime University, Matsuyama, Ehime, Japan\n4Department of Public Health, Graduate School of Medicine, Hokkaido University, Sapporo, Hokkaido, Japan\n5Division of Public Health, Department of Social and Environmental Medicine, Graduate School of Medicine, Osaka University, Suita, Osaka, Japan', 'BACKGROUND :Planetary and human health are highly intertwined; our current food system is associated with high gr

In [29]:
import openai
client = openai.OpenAI(
    api_key="sk-fFbciGZkSxj_S3w4QL6fXA",
    base_url="http://10.75.42.137:4000/" 
)

response = client.chat.completions.create(
    model="GPT-4.1-nano", # model to send to the proxy
    messages = [
        {
            "role": "user",
            "content": "this is a test request"
        }
    ]
)

print(response)

ChatCompletion(id='chatcmpl-C6HpDLf5PJRPfgGmHnzOj1tOBrV2p', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! Your test request has been received. How can I assist you today?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1755614523, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_368a354b49', usage=CompletionUsage(completion_tokens=17, prompt_tokens=12, total_tokens=29, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 

In [43]:
print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! Your test request has been received. How can I assist you today?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))


In [ ]:
for x in attachment_text:
    response = client.chat.completions.create(
    model="GPT-4.1-nano", # model to send to the proxy
    content = "divide the following list into chunks for further analysis: " + x
    messages = [
            {
                "role": "user",
                "content": content
            }
        ]
    )

    print(response)
    break